In [1]:
import os
import os.path as op
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy.ma as ma
from scipy import stats

In [2]:
os.getcwd()

'/Users/mikemckenna/Desktop/image_analysis'

In [6]:
os.chdir('/Users/mikemckenna/Desktop/MPT_data/WHBS_MPT/')

filename = 'P17_14DIV_40nm_cortex_slice_1_vid_3'
traj_file = pd.read_csv('msd_{}.csv'.format(filename))

In [7]:
traj_file[0:5]

,Unnamed: 0,Frame,Track_ID,X,Y,MSDs,Gauss,Quality,SN_Ratio,Mean_Intensity
0,0,0.0,0.0,276.852,1672.469,0.000000,0.000000,5.371,0.432,804.767
1,1,1.0,0.0,277.173,1672.586,0.876146,0.819392,6.315,0.607,809.837
2,2,2.0,0.0,276.931,1672.323,0.880650,0.829152,4.790,0.398,802.783
3,3,3.0,0.0,276.843,1672.450,0.932555,0.799256,6.990,0.722,815.589
4,4,4.0,0.0,277.044,1671.872,0.900072,0.720480,5.660,0.601,806.341


In [8]:
particles = int(max(traj_file['Track_ID']))

In [9]:
particles

5055

In [10]:
def unmask_track(track):
    xpos = ma.masked_invalid(track['X'])
    msds = ma.masked_invalid(track['MSDs'])
    x_mask = ma.getmask(xpos)
    msd_mask = ma.getmask(msds)
    comp_frame = ma.compressed(ma.masked_where(msd_mask, track['Frame']))
    compid = ma.compressed(ma.masked_where(msd_mask, track['Track_ID']))
    comp_x = ma.compressed(ma.masked_where(x_mask, track['X']))
    comp_y = ma.compressed(ma.masked_where(x_mask, track['Y']))
    comp_msd = ma.compressed(ma.masked_where(msd_mask, track['MSDs']))
    comp_gauss = ma.compressed(ma.masked_where(msd_mask, track['Gauss']))
    comp_qual = ma.compressed(ma.masked_where(x_mask, track['Quality']))
    comp_snr = ma.compressed(ma.masked_where(x_mask, track['SN_Ratio']))
    comp_meani = ma.compressed(ma.masked_where(x_mask,
                                               track['Mean_Intensity']))

    data1 = {'Frame': comp_frame,
             'Track_ID': compid,
             'X': comp_x,
             'Y': comp_y,
             'MSDs': comp_msd,
             'Gauss': comp_gauss,
             'Quality': comp_qual,
             'SN_Ratio': comp_snr,
             'Mean_Intensity': comp_meani
             }
    comp_track = pd.DataFrame(data=data1)
    return comp_track

In [12]:
single_track = unmask_track(traj_file)

In [14]:
single_track

,Frame,Track_ID,X,Y,MSDs,Gauss,Quality,SN_Ratio,Mean_Intensity
0,0.0,0.0,276.852,1672.469,0.000000,0.000000,5.371,0.432,804.767
1,1.0,0.0,277.173,1672.586,0.876146,0.819392,6.315,0.607,809.837
2,2.0,0.0,276.931,1672.323,0.880650,0.829152,4.790,0.398,802.783
3,3.0,0.0,276.843,1672.450,0.932555,0.799256,6.990,0.722,815.589
4,4.0,0.0,277.044,1671.872,0.900072,0.720480,5.660,0.601,806.341
...,...,...,...,...,...,...,...,...,...
259926,10.0,5055.0,2019.084,36.636,12.873448,0.553438,8.644,0.662,773.116
259927,11.0,5055.0,2013.978,36.428,5.423430,0.798856,5.388,0.757,762.140
259928,12.0,5055.0,2018.931,38.073,28.428776,0.428276,7.825,0.965,772.907
259929,13.0,5055.0,2018.250,35.090,41.429646,0.437087,6.137,0.729,767.008


In [15]:
traj_file.shape

(3291456, 10)

In [16]:
single_track.shape

(259931, 9)

In [17]:
trackids = single_track.Track_ID.unique()

In [18]:
trackids

array([0.000e+00, 1.000e+00, 2.000e+00, ..., 5.053e+03, 5.054e+03,
       5.055e+03])

In [ ]:
for particle in range(0, trackids.shape[0]):
    corrected